In [4]:
import cv2, time, pandas as pd
from datetime import datetime

In [3]:
video=cv2.VideoCapture(0)

first_frame=None
status_list=[None,None]
times=[]
df=pd.DataFrame(columns=['Start','End'])
while True:

    check, frame =video.read()
    status=0
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    gray=cv2.GaussianBlur(gray,(21,21),0)#smooths image to remove noise
    #21,21 is parameteros of blurryness, 0 is std dev
    if first_frame is None:
        first_frame=gray
        continue #means return to the beginning of the loop and do not run rest of code
        
    #depth/difference of frame
    delta_frame=cv2.absdiff(first_frame,gray)
    
    #classifying values as white =255, black=0
    thresh_frame=cv2.threshold(delta_frame,30,255,cv2.THRESH_BINARY)[1]
    
    #smooth threshold frame
    thresh_frame=cv2.dilate(thresh_frame,None,iterations=2)
    
    #contour detection
    (_,cnts,_)=cv2.findContours(thresh_frame.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    for i in cnts:
        if cv2.contourArea(i) <1000:
            continue
        status=1
        
        (x,y,w,h)=cv2.boundingRect (i)
        cv2.rectangle(frame,(x,y),(x+w, y+h),(0,255,0),3)
    #recording datime    
    status_list.append(status)
    if status_list[-1]==1 and status_list[-2]==0:#if changes from 0 to 1
        times.append(datetime.now())
    if status_list[-1]=-1 and status_list[-2]==1:# if changes from 1 to 0 
        times.append(datetime.now())
    
    cv2.imshow('Gray Frame',gray)
    cv2.imshow('Delta Frame',delta_frame)
    cv2.imshow('Threshold Frame',thresh_frame)
    cv2.imshow('Color Frame',frame)
    key=cv2.waitKey(2000)
    
    if key==ord('q'):#press q to exit video
        if staus==1:
        times.append(datetime.now())
        break
print(status_list)
print(times)

for i in range(0,len(times),2):
    df=df.append({"Start":times[i],'End':times[i+1]},ignore_index=True)

df.to_csv('Times.csv')    
video.release()
cv2.destroyAllWindows()

[1, 1]
[datetime.datetime(2018, 5, 21, 14, 26, 36, 846009), datetime.datetime(2018, 5, 21, 14, 26, 49, 769755)]
